Isaac Gregory and Bibek Lamsal

**House Pricing** *for SWD Course*

Noisy Feature Analysis

##Imports and Dataset

In [ ]:
from google.colab import files
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import copy as cp

In [ ]:
# Import kaggle.json file
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# Load dataset
!kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 63.6MB/s]


In [ ]:
# Unzipping data from file
!unzip house-prices-advanced-regression-techniques.zip

Archive:  house-prices-advanced-regression-techniques.zip
  inflating: data_description.txt    
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# Reading in the CSV
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


##Preprocessing

In [ ]:
def CategorizeStringColumns(og_data, encoder_list):
  # Copying so that the original data object isn't overwritten
  data = cp.deepcopy(og_data)

  # Splitting the numerical and categorical features
  cat_cols = data.select_dtypes(include=['object']).columns.tolist()
  num_cols = data.select_dtypes(include=['number']).columns.tolist()

  # Ensuring there are no overlapping features
  shared_cols = list(set(cat_cols) & set(num_cols))
  if shared_cols != None and shared_cols != []:
    print("Shared:", shared_cols)
    return None

  # Encoding all categorical features
  for col in cat_cols:
      encoder = LabelEncoder()
      data[col] = encoder.fit_transform(data[col])
      encoder_list[col] = encoder

  return data

# Example usage:
encoders = {}
encoded_data = CategorizeStringColumns(data, encoders)
print(encoded_data)
print(data)

        Id  MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  \
0        1          60         3         65.0     8450       1      2   
1        2          20         3         80.0     9600       1      2   
2        3          60         3         68.0    11250       1      2   
3        4          70         3         60.0     9550       1      2   
4        5          60         3         84.0    14260       1      2   
...    ...         ...       ...          ...      ...     ...    ...   
1455  1456          60         3         62.0     7917       1      2   
1456  1457          20         3         85.0    13175       1      2   
1457  1458          70         3         66.0     9042       1      2   
1458  1459          20         3         68.0     9717       1      2   
1459  1460          20         3         75.0     9937       1      2   

      LotShape  LandContour  Utilities  ...  PoolArea  PoolQC  Fence  \
0            3            3          0  ...        

##Noise Analysis

In [ ]:
# Separate labels from samples


# Find and graph correlation between all features and the labels
  # Loop through features
  # Calculate correlation
  # Save correlation into array
  # Graph correlation

  # Save each graph into an image file

# Observe features removed using threshold on array

In [ ]:
# Feature to Feature noise comparison


In [ ]:
# Analyze feature's individual distributions
